# 3D Model Viewer

This notebook loads and visualizes 3D models in .OBJ, .STL, and .GLTF formats using trimesh and vedo libraries.

## 1. Import Libraries

In [25]:
import trimesh
import vedo

vedo.settings.default_backend = "vtk"

## 2. Load 3D Model Function

This function loads models in .OBJ, .STL, and .GLTF formats using trimesh.

In [26]:
def load_model(file_path):
    """
    Load a 3D model from a file.
    
    Parameters:
    file_path : str
        Path to the model file (.obj, .stl, .gltf, .glb)
    
    trimesh.Trimesh : The loaded model
    """
    try:
        # Load the model using trimesh
        model = trimesh.load(file_path)
        
        # If the model is a scene (like GLTF), extract the geometry
        if isinstance(model, trimesh.Scene):
            # Combine all geometries into a single mesh
            model = trimesh.util.concatenate(
                tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
                      for g in model.geometry.values())
            )
        
        print(f"Model loaded successfully: {file_path}")
        return model
    
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

## 3. Display Structural Information

This function displays the number of vertices, edges, and faces of the model.

In [27]:
def show_model_info(model):
    """
    Display structural information of the 3D model.
    
    Parameters:
    model : trimesh.Trimesh
        The 3D model to analyze
    """
    if model is None:
        print("No model loaded")
        return
    
    # Basic information
    print(f"\nMesh Structure:")
    print(f"  • Number of vertices: {len(model.vertices):,}")
    print(f"  • Number of faces: {len(model.faces):,}")
    print(f"  • Number of edges: {len(model.edges):,}")

## 4. Visualization with Vedo

This function creates an interactive visualization.

In [28]:
def visualize_model_vedo(model, title="3D Model"):
    """
    Visualize the 3D model using vedo with different representations.
    
    Parameters:
    model : trimesh.Trimesh
        The 3D model to visualize
    title : str
        Visualization title
    """
    if model is None:
        print("No model loaded")
        return
    
    # Convert trimesh to vedo mesh
    mesh_vedo = vedo.Mesh([model.vertices, model.faces])
    
    # Create multiple representations
    # 1. Faces with solid color (light blue)
    mesh_faces = mesh_vedo.clone().color('lightblue').lighting('glossy')
    
    # 2. Edges (wireframe) in red
    mesh_edges = mesh_vedo.clone().wireframe().color('red').linewidth(2)
    
    # 3. Vertices in yellow
    vertices_points = vedo.Points(model.vertices).color('yellow').point_size(8)
    
    # Create plotter: two columns on top, centered single view below
    plt_vedo = vedo.Plotter(shape=(2, 2), title=title, size=(1400, 1000))
    for renderer in plt_vedo.renderers:
        renderer.SetBackground(0.10, 0.10, 0.10)
    
    # Top row
    plt_vedo.at(0).show(mesh_faces, "Faces", axes=1, viewup='z')
    plt_vedo.at(1).show(mesh_edges, "Edges", axes=1, viewup='z')

    # Bottom row: center the left renderer and hide the right renderer
    plt_vedo.renderers[2].SetViewport(0.25, 0.0, 0.75, 0.5)
    plt_vedo.renderers[3].SetViewport(0.0, 0.0, 0.0, 0.0)
    plt_vedo.at(2).show(vertices_points, "Vertices", axes=1, viewup='z')

    plt_vedo.show(interactive=True)
    return plt_vedo


## 5. Load and Visualize a Model

Load a model and display its structural information.

In [29]:
# Path to the model
model_path = "torus.glb"  # Can be .obj, .stl, .gltf, or .glb

# Load the model
model = load_model(model_path)

# Display structural information
if model is not None:
    show_model_info(model)

Model loaded successfully: torus.glb

Mesh Structure:
  • Number of vertices: 1,600
  • Number of faces: 3,200
  • Number of edges: 9,600


## 6. Interactive Visualization 

This cell opens an interactive window and colors faces, edges, and vertices separately.

In [ ]:
if model is not None:
    visualize_model_vedo(model)